# CSCI 3202: Introduction to Artificial Intelligence, Fall 2023
## Final Mancala Project
### Your name: John Le
​
<br> 

#### Code from Hw05

## Interface to Play Mancala and Random Player

In [1]:
import random
import numpy as np

In [2]:
class Mancala:
    def __init__(self, pits_per_player=6, stones_per_pit = 4):
        """
        The constructor for the Mancala class defines several instance variables:

        pits_per_player: This variable stores the number of pits each player has.
        stones_per_pit: It represents the number of stones each pit contains at the start of any game.
        board: This data structure is responsible for managing the Mancala board.
        current_player: This variable takes the value 1 or 2, as it's a two-player game, indicating which player's turn it is.
        moves: This is a list used to store the moves made by each player. It's structured in the format (current_player, chosen_pit).
        p1_pits_index: A list containing two elements representing the start and end indices of player 1's pits in the board data structure.
        p2_pits_index: Similar to p1_pits_index, it contains the start and end indices for player 2's pits on the board.
        p1_mancala_index and p2_mancala_index: These variables hold the indices of the Mancala pits on the board for players 1 and 2, respectively.
        """
        self.pits_per_player = pits_per_player
        self.stones_per_pit = stones_per_pit
        self.board = [stones_per_pit] * ((pits_per_player+1) * 2)  # Initialize each pit with stones_per_pit number of stones 
        self.players = 2
        self.current_player = 1
        self.moves = []
        self.p1_pits_index = [0, self.pits_per_player-1]
        self.p1_mancala_index = self.pits_per_player
        self.p2_pits_index = [self.pits_per_player+1, len(self.board)-1-1]
        self.p2_mancala_index = len(self.board)-1
        
        # Zeroing the Mancala for both players
        self.board[self.p1_mancala_index] = 0
        self.board[self.p2_mancala_index] = 0

    def display_board(self):
        """
        Displays the board in a user-friendly format
        """
        player_1_pits = self.board[self.p1_pits_index[0]: self.p1_pits_index[1]+1]
        player_1_mancala = self.board[self.p1_mancala_index]
        player_2_pits = self.board[self.p2_pits_index[0]: self.p2_pits_index[1]+1]
        player_2_mancala = self.board[self.p2_mancala_index]

        print('P1               P2')
        print('     ____{}____     '.format(player_2_mancala))
        for i in range(self.pits_per_player):
            if i == self.pits_per_player - 1:
                print('{} -> |_{}_|_{}_| <- {}'.format(i+1, player_1_pits[i], 
                        player_2_pits[-(i+1)], self.pits_per_player - i))
            else:    
                print('{} -> | {} | {} | <- {}'.format(i+1, player_1_pits[i], 
                        player_2_pits[-(i+1)], self.pits_per_player - i))
            
        print('         {}         '.format(player_1_mancala))
        turn = 'P1' if self.current_player == 1 else 'P2'
        print('Turn: ' + turn)
        
    def valid_move(self, pit):
        """
        Function to check if the pit chosen by the current_player is a valid move.
        """
        
        # write your code here
        start_index, end_index = (
            self.p1_pits_index if self.current_player == 1 else self.p2_pits_index
        )
        if pit < 1 or pit > self.pits_per_player or self.board[start_index + pit - 1] == 0:
            #print("INVALID MOVE")
            return False
        return True
    
        pass
        
    def random_move_generator(self):
        """
        Function to generate random valid moves with non-empty pits for the random player
        """
        
        pit = random.randint(1, self.pits_per_player)
        
        while not self.valid_move(pit):
            pit = random.randint(1, self.pits_per_player)
            
        return pit
    
    def play(self, pit):
        """
        This function simulates a single move made by a specific player using their selected pit. It primarily performs three tasks:
        1. It checks if the chosen pit is a valid move for the current player. If not, it prints "INVALID MOVE" and takes no action.
        2. It verifies if the game board has already reached a winning state. If so, it prints "GAME OVER" and takes no further action.
        3. After passing the above two checks, it proceeds to distribute the stones according to the specified Mancala rules.

        Finally, the function then switches the current player, allowing the other player to take their turn.
        """
        
        # write your code here
        if not self.valid_move(pit) or self.winning_eval():
            #print(f"Player {self.current_player} chose pit: {pit}")
            return self.board

        start_index, end_index = (
            self.p1_pits_index if self.current_player == 1 else self.p2_pits_index
        )
        stones = self.board[start_index + pit - 1]
        self.board[start_index + pit - 1] = 0

        current_index = start_index + pit
        while stones > 0:
            if current_index == self.p1_mancala_index and self.current_player == 2:
                current_index += 1
            elif current_index == self.p2_mancala_index and self.current_player == 1:
                current_index = start_index
            else:
                self.board[current_index] += 1
                stones -= 1
                current_index = (current_index + 1) % len(self.board)

        # Check for capturing on the current player's side
        last_index = (current_index - 1) % len(self.board)
        opposite_pit_index = len(self.board) - 2 - last_index
        current_player_pits_start, current_player_pits_end = (
            self.p1_pits_index if self.current_player == 1 else self.p2_pits_index
        )
        if (
            last_index != self.p1_mancala_index
            and last_index != self.p2_mancala_index
            and current_player_pits_start <= last_index <= current_player_pits_end
            and self.board[last_index] == 1
            and self.board[opposite_pit_index] > 0
        ):
            self.board[self.p1_mancala_index if self.current_player == 1 else self.p2_mancala_index] += (
                self.board[last_index] + self.board[opposite_pit_index]
            )
            self.board[last_index] = 0
            self.board[opposite_pit_index] = 0

        # Switch player after move
        self.current_player = 3 - self.current_player  # Switch between player 1 and player 2

        # Save the move
        self.moves.append((3 - self.current_player, pit))
        
        #print(f"Player {3 - self.current_player} chose pit: {pit}")

        return self.board
    
    def evaluate_state(self):
        return self.board[self.p1_mancala_index] - self.board[self.p2_mancala_index]
    
    def winning_eval(self):
        """
        Function to verify if the game board has reached the winning state.
        Hint: If either of the players' pits are all empty, then it is considered a winning state.
        """
        
        # write your code here
        if all(self.board[i] == 0 for i in range(self.p1_pits_index[0], self.p1_pits_index[1] + 1)):
            self.board[self.p2_mancala_index] += sum(self.board[self.p2_pits_index[0] : self.p2_pits_index[1] + 1])
            return True
        elif all(self.board[i] == 0 for i in range(self.p2_pits_index[0], self.p2_pits_index[1] + 1)):
            self.board[self.p1_mancala_index] += sum(self.board[self.p1_pits_index[0] : self.p1_pits_index[1] + 1])
            return True
        return False
    
        pass
    
    def generate_board(self):
        self.board = [self.stones_per_pit] * ((self.pits_per_player + 1) * 2)
        self.current_player = 1
        self.moves = []
        self.board[self.p1_mancala_index] = 0
        self.board[self.p2_mancala_index] = 0
    
    def match_analysis(self, games=100, ai_player=None, alpha_beta=False):
        self.p1_wins = 0
        self.p2_wins = 0
        self.ties = 0
        total_moves = 0

        for _ in range(games):
            self.generate_board()
            moves_in_current_game = 0

            while not self.winning_eval():
                if ai_player and self.current_player == 2:
                    move = ai_player.best_move(self, alpha_beta)
                else:
                    move = self.random_move_generator()
                self.play(move)
                moves_in_current_game += 1

            total_moves += moves_in_current_game

            if self.board[self.p1_mancala_index] > self.board[self.p2_mancala_index]:
                self.p1_wins += 1
            elif self.board[self.p1_mancala_index] < self.board[self.p2_mancala_index]:
                self.p2_wins += 1
            else:
                self.ties += 1

        average_moves = total_moves / games
        print(f"P1 Wins: {self.p1_wins}, P2 Wins: {self.p2_wins}, Ties: {self.ties}")
        print(f"Average moves to win: {average_moves}")

### Displays Board with Pre-moves determined (uncomment to see)

In [3]:
# #Mancala part 1 

# game = Mancala()
# game.display_board()

# # Player 1 selects pit 1 (1-based index)
# game.play(1)
# game.display_board()

# # Player 2 selects pit 2
# game.play(2)
# game.display_board()

# # Player 1 selects pit 3
# game.play(3)
# game.display_board()

# # Player 2 selects pit 2
# game.play(2)
# game.display_board()

# # Player 1 selects pit 1
# game.play(1)
# game.display_board()

# # Printing the list of moves
# print("\nList of valid moves:")
# for move in game.moves:
#     player, pit = move
#     print(f"Player {player} selected pit {pit}")


### Generate Game against a Random Player (uncomment to play)

In [4]:
# # Mancala part 2 

# game = Mancala(pits_per_player=6, stones_per_pit=4)
# game.display_board()

# # Play up to 5 moves by the player and 5 moves by the random player
# total_moves = 10
# for _ in range(total_moves):
#     if game.current_player == 1:
#         # Player's move
#         player_move = int(input())
#         game.play(player_move)
#     else:
#         # Random player's move
#         random_move = game.random_move_generator()
#         game.play(random_move)

#     game.display_board()

#     if game.winning_eval():
#         print("GAME OVER")
#         break 

# # Printing the list of moves
# print("\nList of valid moves:")
# for move in game.moves:
#     player, pit = move
#     print(f"Player {player} selected pit {pit}")

## Implementing AI to Play Mancala (new code)

In [5]:
import copy, math, time, random
random.seed(109)

#### Random player vs. Random Player

In [6]:
# Create an instance of the Mancala class
game = Mancala()

# Play 100 games of random player against random player
game.match_analysis(games=100)

P1 Wins: 44, P2 Wins: 48, Ties: 8
Average moves to win: 43.76


3. Play 100 games of random player against random player
    - What percentage of games does each player (1st or 2nd) win?
        - `On average, for a random vs random simulation, the percentage that each player wins ranges from 47-50% and around a 2-3% tie rate. The wins for each player are close to being a 50-50 split which is what I believe should be happening.`
    - On average, how many moves does it take to win?
        - `On average, it ranges around 42-46 moves to win a game.`

# Intermediate Results

As of today, I thought that I had completed the initial phase of the project, but it turned out I had an error in my **'random_move_generator'** that made the random player stuck on one invalid move infinitely after a certain amount of moves. I managed to figure it out and now I am working on the next step of the project which is to immplement the minimax and Alpha-Beta players. Everything so far seems to work on my project, I just haven't tested the minimax player or implemented the other yet. And since hw05, I have just been trying to figure out why the error I previously mentioned was happening and I figured it out a bit too late for this Intermediate Results deadline. I haven't done that much outside of hw05 but I will get a lot more done in the coming week before the deadline of the project. 

## Mancala Minimax AI

In [12]:
class MancalaAI:
    def __init__(self, depth, state):
        # Constructor to initialize the AI with a given search depth and game state
        self.depth = depth
        self.state = state
    
    # Define a minimax function for game-playing AI
    def minimax(self, state, depth, maximizing_player, cur_player):
        # Base case: if the depth is 0 or the current state is a winning state, return the evaluation of the state
        if depth == 0 or state.winning_eval():
            return self.evaluate_state(state)

        # Get a list of possible valid moves for the current state
        possible_valid_moves = self.get_valid_moves(state)
        
        # If it's the turn of the maximizing player
        if maximizing_player:
            # Initialize max_eval to negative infinity
            max_eval = float('-inf')
            # Iterate through each possible move
            for move in possible_valid_moves:
                # Simulate the move and get the new state
                new_state = self.simulate_move(state, move)
                # Recursively call minimax with the new state, decreasing depth, and switching players
                eval = self.minimax(new_state, depth - 1, False, 3 - cur_player)
                # Update max_eval with the maximum value between max_eval and eval
                max_eval = max(max_eval, eval)
            # Return the maximum evaluation for the maximizing player's turn
            return max_eval
        # If it's the turn of the minimizing player
        else:
            # Initialize min_eval to positive infinity
            min_eval = float('inf')
            # Iterate through each possible move
            for move in possible_valid_moves:
                # Simulate the move and get the new state
                new_state = self.simulate_move(state, move)
                # Recursively call minimax with the new state, decreasing depth, and switching players
                eval = self.minimax(new_state, depth - 1, True, 3 - cur_player)
                # Update min_eval with the minimum value between min_eval and eval
                min_eval = min(min_eval, eval)
            # Return the minimum evaluation for the minimizing player's turn
            return min_eval
    
    # Define a minimax algorithm with alpha-beta pruning for game-playing AI
    def minimax_alpha_beta(self, state, depth, alpha, beta, maximizing_player, cur_player):
        # Base case: if the depth is 0 or the current state is a winning state, return the evaluation of the state
        if depth == 0 or state.winning_eval():
            return self.evaluate_state(state)

        # Get a list of possible valid moves for the current state
        possible_valid_moves = self.get_valid_moves(state)

        # If it's the turn of the maximizing player
        if maximizing_player:
            # Initialize max_eval to negative infinity
            max_eval = float('-inf')
            # Iterate through each possible move
            for move in possible_valid_moves:
                # Simulate the move and get the new state
                new_state = self.simulate_move(state, move)
                # Recursively call minimax_alpha_beta with the new state, decreasing depth, and switching players
                eval = self.minimax_alpha_beta(new_state, depth - 1, alpha, beta, False, 3 - cur_player)
                # Update max_eval with the maximum value between max_eval and eval
                max_eval = max(max_eval, eval)
                # Update alpha with the maximum value between alpha and eval
                alpha = max(alpha, eval)
                # If beta is less than or equal to alpha, prune the search and break out of the loop
                if beta <= alpha:
                    break
            # Return the maximum evaluation for the maximizing player's turn
            return max_eval
        # If it's the turn of the minimizing player
        else:
            # Initialize min_eval to positive infinity
            min_eval = float('inf')
            # Iterate through each possible move
            for move in possible_valid_moves:
                # Simulate the move and get the new state
                new_state = self.simulate_move(state, move)
                # Recursively call minimax_alpha_beta with the new state, decreasing depth, and switching players
                eval = self.minimax_alpha_beta(new_state, depth - 1, alpha, beta, True, 3 - cur_player)
                # Update min_eval with the minimum value between min_eval and eval
                min_eval = min(min_eval, eval)
                # Update beta with the minimum value between beta and eval
                beta = min(beta, eval)
                # If beta is less than or equal to alpha, prune the search and break out of the loop
                if beta <= alpha:
                    break
            # Return the minimum evaluation for the minimizing player's turn
            return min_eval

    # Define a function to find the best move for the current player
    def best_move(self, state, alpha_beta=False):
        # Get a list of possible valid moves for the current state
        possible_valid_moves = self.get_valid_moves(state)
        # Initialize variables to store the best move and its corresponding value
        best_move = None
        best_value = float('-inf') if state.current_player == 1 else float('inf')

        # Iterate through each possible move
        for move in possible_valid_moves:
            # Simulate the move and get the new state
            new_state = self.simulate_move(state, move)
            # Use either minimax with alpha-beta pruning or basic minimax to get the value of the move
            value = self.minimax_alpha_beta(new_state, self.depth, float('-inf'), float('inf'), not state.current_player == 1, state.current_player) if alpha_beta else self.minimax(new_state, self.depth, not state.current_player == 1, state.current_player)

            # Compare the obtained value with the best value based on the current player
            if (state.current_player == 1 and value > best_value) or (state.current_player == 2 and value < best_value):
                # If the obtained value is better, update the best value and move
                best_value = value
                best_move = move

        # Return the best move found
        return best_move

    # Define a function to evaluate the given game state
    def evaluate_state(self, state):
        # Basic scoring based on Mancala count
        score = state.board[state.p1_mancala_index] - state.board[state.p2_mancala_index]

        # Factor in potential captures and board control
        for i in range(state.pits_per_player):
            # Evaluate player 1's side
            if state.board[i] == (state.pits_per_player - i):
                score += 2  # Potential capture or consecutive turn
            if state.board[i] > 0:
                score += 1  # Board control

            # Evaluate player 2's side
            opp_index = state.p2_pits_index[0] + i
            if state.board[opp_index] == (state.pits_per_player - i):
                score -= 2
            if state.board[opp_index] > 0:
                score -= 1

        return score

    # Define a function to get valid moves for the current state
    def get_valid_moves(self, state):
        # Helper function to get valid moves for the current state
        valid_moves = []
        # Iterate through each pit for the current player
        for i in range(1, state.pits_per_player + 1):
            # Check if the move is valid for the current pit
            if state.valid_move(i):
                # Add the valid move to the list
                valid_moves.append(i)
        # Return the list of valid moves
        return valid_moves

    # Define a function to simulate a move without modifying the original state
    def simulate_move(self, state, move):
        # Helper function to simulate a move without modifying the original state
        # Create a deep copy of the current state to avoid modifying the original state
        new_state = copy.deepcopy(state)
        # Simulate the move on the copied state
        new_state.play(move)
        # Return the modified state after the simulated move
        return new_state

#### Minimax Player vs. Random Player

In [13]:
# Create an instance of the Mancala class
game = Mancala()

# Create an instance of the Minimax AI player with a depth of 5 plies
ai_player = MancalaAI(depth=5, state=game)

# Play 100 games with the random player against the Minimax AI player
game.match_analysis(games=100, ai_player=ai_player)

# Display the results

P1 Wins: 1, P2 Wins: 99, Ties: 0
Average moves to win: 32.92


4. Build an AI player that uses minimax to choose the best move with a variable number of plies and a utility function we describe
    - What percentage of games does each player (AI or random) win?
        - `The AI player has a win percentage of over 80% against a random player.`
    - On average, how many moves does it take to win?
        - `On average, it ranges around 31-34 moves to win a game.`
5. Play 100 games with the random player against the minimax AI player at a depth of 5 plies
    - What percentage of games does each player (AI or random) win?
        - `On average, for a minimax vs random simulation, the percentage that the minimax player wins ranges from ~98-100%, around a ~1-2% random player winrate, and a ~1% tie rate. The minimax player should win most of the time and the random player may win a handful of times.`
    - On average, how many moves does it take to win?
        - `On average, it ranges around 30-32 moves to win a game.`
    - Is your AI player better than random chance? Write a paragraph or two describing or why not
        - `The performance of the Minimax AI player, with a 99% win rate against the random player, clearly indicates that it is significantly better than random chance. This outcome is not surprising given the nature of the Minimax algorithm, which systematically evaluates possible future moves and their outcomes to make the best decision at each turn.`

            `In contrast, a random player lacks strategic decision-making and simply chooses moves without considering their implications, leading to less optimized play. The Minimax algorithm's ability to look ahead and evaluate game states based on a utility function gives it a substantial advantage. This is evident from the high win rate and the relatively consistent number of moves required to win a game.`

            `The fact that the AI wins the vast majority of games and does so with a fairly consistent number of moves suggests that the AI is not only exploiting the random player's lack of strategy but is also efficiently navigating towards winning end-game scenarios. This efficiency reflects the effectiveness of the utility function and the depth of the lookahead used in the algorithm. It demonstrates that the Minimax AI is making moves that consistently lead to advantageous positions, ultimately culminating in wins far more often than what would be expected by chance.`  

## Alpha-Beta AI

In [15]:
class MancalaAI:
    def __init__(self, depth, state):
        self.depth = depth
        self.state = state

    # Define a minimax function for game-playing AI (same as above)
    def minimax(self, state, depth, maximizing_player, cur_player):
        if depth == 0 or state.winning_eval():
            return self.evaluate_state(state)

        possible_valid_moves = self.get_valid_moves(state)
        
        if maximizing_player:
            max_eval = float('-inf')
            for move in possible_valid_moves:
                new_state = self.simulate_move(state, move)
                eval = self.minimax(new_state, depth - 1, False, 3 - cur_player)
                max_eval = max(max_eval, eval)
            return max_eval
        else:
            min_eval = float('inf')
            for move in possible_valid_moves:
                new_state = self.simulate_move(state, move)
                eval = self.minimax(new_state, depth - 1, True, 3 - cur_player)
                min_eval = min(min_eval, eval)
            return min_eval

    # New Alpha-Beta Pruning method
    def alpha_beta_pruning(self, state, depth, alpha, beta, maximizing_player):
        if depth == 0 or state.winning_eval():
            return self.evaluate_state(state)

        if maximizing_player:
            max_eval = float('-inf')
            for move in self.get_valid_moves(state):
                new_state = self.simulate_move(state, move)
                eval = self.alpha_beta_pruning(new_state, depth - 1, alpha, beta, False)
                max_eval = max(max_eval, eval)
                alpha = max(alpha, eval)
                if beta <= alpha:
                    break
            return max_eval
        else:
            min_eval = float('inf')
            for move in self.get_valid_moves(state):
                new_state = self.simulate_move(state, move)
                eval = self.alpha_beta_pruning(new_state, depth - 1, alpha, beta, True)
                min_eval = min(min_eval, eval)
                beta = min(beta, eval)
                if beta <= alpha:
                    break
            return min_eval

    # Updated best_move to use Alpha-Beta Pruning
    def best_move(self, state, alpha_beta=False):
        possible_valid_moves = self.get_valid_moves(state)
        best_move = None
        best_value = float('-inf') if state.current_player == 1 else float('inf')

        for move in possible_valid_moves:
            new_state = self.simulate_move(state, move)
            value = None
            if alpha_beta:
                value = self.alpha_beta_pruning(new_state, self.depth, float('-inf'), float('inf'), not state.current_player == 1)
            else:
                value = self.minimax(new_state, self.depth, not state.current_player == 1, state.current_player)

            if (state.current_player == 1 and value > best_value) or (state.current_player == 2 and value < best_value):
                best_value = value
                best_move = move

        return best_move

    # Define a function to evaluate the given game state
    def evaluate_state(self, state):
        # Basic scoring based on Mancala count
        score = state.board[state.p1_mancala_index] - state.board[state.p2_mancala_index]

        # Factor in potential captures and board control
        for i in range(state.pits_per_player):
            # Evaluate player 1's side
            if state.board[i] == (state.pits_per_player - i):
                score += 2  # Potential capture or consecutive turn
            if state.board[i] > 0:
                score += 1  # Board control

            # Evaluate player 2's side
            opp_index = state.p2_pits_index[0] + i
            if state.board[opp_index] == (state.pits_per_player - i):
                score -= 2
            if state.board[opp_index] > 0:
                score -= 1

        return score

    # Define a function to get valid moves for the current state (same as above)
    def get_valid_moves(self, state):
        # Helper function to get valid moves for the current state
        valid_moves = []
        for i in range(1, state.pits_per_player + 1):
            if state.valid_move(i):
                valid_moves.append(i)
        return valid_moves

    # Define a function to simulate a move without modifying the original state (same as above)
    def simulate_move(self, state, move):
        # Helper function to simulate a move without modifying the original state
        new_state = copy.deepcopy(state)
        new_state.play(move)
        return new_state


#### Alpha-Beta Player vs. Random Player

In [10]:
# Test Alpha-Beta AI against Random Player
game = Mancala()
alpha_beta_ai = MancalaAI(depth=5, state=game)

# Use alpha_beta=True to engage the Alpha-Beta Pruning AI
game.match_analysis(games=100, ai_player=alpha_beta_ai, alpha_beta=True)

P1 Wins: 0, P2 Wins: 99, Ties: 1
Average moves to win: 31.08


6. Build an AI player that uses Alpha-Beta to choose the best move
7. Play 100 games with the random player against the Alpha-Beta AI player at a depth of 5 plies
    - How long does it take for a single game to run to completion?
        - `On average, it only takes around 1m 40s for all 100 games to be completed. This would mean that it takes around 1s to complete a single game.`
    - What percentage of games does each player (AI or random) win?
        - `Similar to the Minimax player, the Alpha-Beta Player has a win percentage from between ~98-100% while the Random player will rarely win at a percentage of ~1-2%. The tie rate is also ~1% as well.`
    - On average, how many moves does it take to win?
        - `On average, it still takes around 30-32 moves to win a game of Mancala.`
    - Are your results for this part different from those for your minimax AI player? Write a paragraph or two describing why or why not?
        - `The win rates for both the Minimax and Alpha-Beta algorithms are identical, with the AI winning 99% of games in both cases. This similarity is expected as Alpha-Beta pruning and Minimax are fundamentally the same in terms of decision-making and strategy. They both use the same utility function and decision criteria; the only difference lies in their efficiency.`

            `However, a significant difference is observed in the runtime. The Alpha-Beta algorithm shows a marked improvement in efficiency, with an average game taking considerably less time (1 minute and 40 seconds) compared to the Minimax algorithm (6 minutes and 45 seconds). This difference is due to the pruning mechanism of the Alpha-Beta algorithm, which effectively skips evaluating parts of the game tree that cannot affect the final decision. This reduction in the number of evaluated nodes results in faster computation times.`

            `In summary, while the decision-making quality of the AI remains consistent between Minimax and Alpha-Beta, the latter proves to be more time-efficient due to its ability to eliminate irrelevant branches in the game tree. This demonstrates the practical advantage of Alpha-Beta pruning in scenarios where computation time is a critical factor.`

#### Alpha-Beta Player vs. Random Player (10plies) [actually 6 because 10 took way to long and it wasn't completed]

In [18]:
# Test Alpha-Beta AI against Random Player
game = Mancala()
alpha_beta_ai = MancalaAI(depth=6, state=game)

# Use alpha_beta=True to engage the Alpha-Beta Pruning AI
game.match_analysis(games=100, ai_player=alpha_beta_ai, alpha_beta=True)

P1 Wins: 0, P2 Wins: 100, Ties: 0
Average moves to win: 31.59


8. (Extra Credit, 10 points). Play 100 games with the random player against the Alpha-Beta AI player at a depth of 10 plies
    - How long does it take for a single game to run to completion?
        - `It took over 100 minutes and the games still wasn't finished so I had to stop it destroying my laptop. Even when I increased the depth of the plies to 6, the runtime increased from 1m 40s to 5m 20s which is 220% increase so increasing it to 10 plies may actually take a full day to complete.`
    - What percentage of games does each player (AI or random) win?
        - `I would believe that the percentage that the AI would win with a depth of 10 plies would be 100%. The depth of 6 plies already has a 100% win percentage for teh Alpha-beta player so I would assume it would be the same for any depth of plies greater than 5.`
    - On average, how many moves does it take to win?
        - `The average amount of moves to win the game remains the same though, it takes ~31 moves for the Alpha-Beta player to win a game.`
    - Does increasing the number of plies improve the play for our AI player? Why or why not?
        - `Even increasing the number of plies by 1 improves the play for out AI but it does increase the completion time of the games. This happends because the AI has Deeper Strategic Planning, a Better Evaluation of Game States, and Improved Handling of Complex Scenarios. The win percentage increases but the runtime also increases so it is like a double-edged sword.`

## Project Write-up and Summary

### Project Overview

This project encompasses several stages:

`Basic Mancala Interface`: An interface for playing Mancala was developed. This included displaying the game board, prompting for moves, checking for legal moves, and determining game outcomes.

`Random Player`: A player making random legal moves was implemented. This served as a baseline opponent for the AI.

`Minimax and Alpha-Beta AI`: Two AI players were created using Minimax and Alpha-Beta pruning algorithms. These AIs calculated the best moves based on a utility function that evaluated the game state.

<br>

### Testing and Evaluation

The Random Player and AIs were tested in Various different scenarios:

`Random vs Random`: This tested the baseline scenario, with both players making random moves.

Results:
- The win percentage of the random players would be ~50% win rate for each player
- The average amount of moves for them to win was ~42-46 moves
- THe runtime of a 100-game simulation was nearly instant

`Minimax vs Random`: The Minimax AI was against a Random player to test the efficiency of the algorithms developed for Minimax.

Results:
- The win percentage for the Minimax AI was significantly greater than the Random player. With the depth of 5 plies, the win percentage of Minimax ranged from ~98-100% while the Random Player rarely wins, and may occasionally have a tie.
- The average amount of moves to win a game is also less than the Random vs Random. It takes an average of ~31 moves to win a game with the Minimax algorithm.
- The average runtime for the Minimax player vs Random player to complete all 100 games took around 6m 30s. This shows that it takes a while for the Minimax to make decisions to complete the games.

`Alpha-Beta vs Random`: The Alpha-Beta AI was against a Random Player to test the efficiency of the alpha-beta pruning algorithm.

Results:
- The win percentage for the Alpha-Beta player was similar to the Alpha-Beta player which was that the AI, with the depth of 5 plies, had a win percentage that ranged ~98-100% while the Random player would rarely wins, and an occasional tie would happen.
- The average amount of moves to win a game is similar to the Minimax AI. It took an average of ~31 moves to win a game with the Alpha-Beta player as well.
- Unlike the runtime of the Minimax vs Random simulation, the runtime of the Alpha-Beta was significantly less when running it with the depth of 5 plies. The average runtime of the Alpha-Beta simulation took ~1m 40s which means 1s per game played. This is roughly a 75% improvement between the two different AIs. 
(When I ran the Alpha-Beta player with the depth of 10plies, the simulation surpassed 100 minutes which I had to stop)

<br>

### Key Findings and Analysis

`Performance of AI Against Random Player`:
- The Minimax AI significantly outperformed the random player, winning the majority of the games. This was expected as Minimax evaluates potential future game states, while the random player does not strategize.
- The Alpha-Beta AI also demonstrated a high win rate against the random player, similar to the Minimax AI. This similarity is because both algorithms use the same decision-making process, with Alpha-Beta being more efficient in time.

`Efficiency of Alpha-Beta Pruning`:
- A notable difference was observed in the runtime between the Minimax and Alpha-Beta AIs. The Alpha-Beta AI was faster, thanks to its pruning mechanism that skips irrelevant branches in the game tree. This efficiency makes Alpha-Beta a more practical choice in time-sensitive applications.

`AI's Decision-Making Capability`:
- Both AIs showed a consistent ability to reach advantageous positions and navigate towards winning end-game scenarios. This consistency indicated effective use of the utility function and the depth of the lookahead in the algorithms.

`Utility Function and Game Tree Evaluation`:
- The utility function, focusing on the number of stones in each player's Mancala, effectively guided the AI in making strategic decisions. Additionally, the game tree's thorough evaluation by the algorithms ensured that the AI considered all possible outcomes before making a move.

<br>

### Conclusion
The AI application for Mancala successfully demonstrated the power of algorithmic decision-making in board games. The Minimax and Alpha-Beta AIs not only outperformed a random opponent but also did so with a high degree of consistency and efficiency. These findings underline the potential of AI in strategic game-playing scenarios and the effectiveness of algorithms like Minimax and Alpha-Beta pruning in decision-making processes.